In [84]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import emoji
from spellchecker import SpellChecker
import string


# Processar.

Foram coletado 5011 tweets, porém com o processo de classificação do sentimentos dos tweets percebi que haviam textos duplicados e logo foram retirados. Ao total são 4825 tweets classificados para o projeto.  

Com base nas palavras de busca, para dar inicio ao pré-processamento, irei fazer uma cópia da coluna text do df em um novo arquivo data frame para fazer o controle e o comparativo das etapas do processamento do texto.

In [85]:
df_text = pd.read_csv(r'dados\tweets_classificados_raw.csv')
df_text = pd.DataFrame(df_text['texto'].copy())
df_text.head(6)

,texto
0,Gente eu amo os perfumes egeo da boticário
1,"Carai botica 214 da boticário eu te amo ""dark ..."
2,Comprar o Boticário com a minha tia é um perig...
3,@CentralEslo Boticário\nESLU EM CARUARU
4,Uso natura e boticário feliz da vida kkkkkkkkk
5,"Ainda mais que quase ngm tem meu cheiro, impor..."


In [86]:
df_text.shape

(4825, 1)

## Deixando tudo em letra minúcula.

Uma técnica muito comundo no processamento de texto é normalizar as letras para caixa baixa para que palavras iguais não sejam entendidas pelo modelo como palavras diferentes, exemplo: Perfume e perfume.

In [87]:
df_text['texto_minusculo'] = df_text['texto'].str.lower()
df_text.head()

,texto,texto_minusculo
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ..."
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk


## Removendo usuario marcados com @

Irei fazer a remoção de nome de úsuario pois tais informações não ao objeivo do projeto.

In [88]:
#localziadno textos que tem @
df_urls = df_text[df_text['texto_minusculo'].str.contains('@', na = False)]
df_urls.head()

,texto,texto_minusculo
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru
7,@paulo_capeletti Entra no site da Boticário aí...,@paulo_capeletti entra no site da boticário aí...
8,O creme deleite da @boticario é o melhor ❤️,o creme deleite da @boticario é o melhor ❤️
9,"@Antonio09636317 Amigo, tô contigo.\nMês passa...","@antonio09636317 amigo, tô contigo.\nmês passa..."
16,@LulaOficial Ela vende o quê? Jequiti ou Botic...,@lulaoficial ela vende o quê? jequiti ou botic...


In [89]:
def remove_user(text):
    #Procura texto com o padrão especificado
    url_path = re.compile(r'@\S+')
    #substitui
    text_s_url = re.sub(url_path,'',text)
    return text_s_url

#testenado a funçao
text = df_text.texto_minusculo.loc[8]
print(f'Texto original: {text}')
print(f'Texto sem url: {remove_user(text)}')

Texto original: o creme deleite da @boticario é o melhor ❤️
Texto sem url: o creme deleite da  é o melhor ❤️


In [90]:
df_text['texto_sem_arroba'] = df_text['texto_minusculo'].apply(remove_user)
df_text.head()

,texto,texto_minusculo,texto_sem_arroba
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ..."
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk


## Removendo quebra de linhas.

In [91]:
df_text['texto_sem_quebra'] = df_text['texto_sem_arroba'].str.replace('\n',' ')
df_text.head()

,texto,texto_minusculo,texto_sem_arroba,texto_sem_quebra
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ..."
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário eslu em caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk


## Substituindo abreviações.

No processo de classificação de sentimentos anotei quais eram as principais abreviações dos tweets e, apartir disso, montei um dicionário para substituição dessas abreviações. Também padronizei as variações de kkk.

In [92]:
abrevia = {'mto': 'muito','n': 'não', 'nao': 'não', 'fds': 'final de semana', 'fdp': 'filho da puta', 'vdd': 'verdade', 'dmais': 'demais',
           'pct': 'pacote', 'p': 'para', 'homi': 'homem', 'obgd': 'obrigado', 'q': 'que', 'pq': 'porque','rsrs': 'risos','brinks': 'brincadeira',
           'amg': 'amigo','fikdik':'fica dica', 'pqp': 'puta que pariu', 'td': 'tudo', 'tdo': 'tudo', 'ngm': 'ninguém', 'sdds': 'saudades',
           'mn': 'mano', 'gnt': 'gente', 'sqm': 'só que não', 'vc': 'você', 'vcs': 'vocês', 'ss': 'sim', 'grl': 'galera', 'pft': 'perfeito',
           'sm': 'sem', 'flw': 'falou', 'aq': 'aqui', 'bnt': 'bonito', 'cm': 'com', 'pprt': 'papo reto', 'pfv': 'por favor', 'fml': 'familia',
           'obg': 'obrigado','k':'kkk', 'kk': 'kkk', 'kkkk': 'kkk', 'kkkkk': 'kkk', 'kkkkkk':'kkk', 'kkkkkkk':'kkk', 'kkkkkkkk':'kkk', 'kkkkkkkkk':'kkk',
            'kkkkkkkkkk':'kkk','kkkkkkkkkkk':'kkk', 'kkkkkkkkkkkk':'kkk','kkkkkkkkkkkkk':'kkk','kkkkkkkkkkkkkk':'kkk' ,'pra':'para', 'eh':'é', 'to':'estou',
              'tô':'estou','boticario':'boticário', 'nd':'nada', 'amoo':'amo', 'amooo':'amo', 'amoooo':'amo'
           }

In [93]:
texto = 'rsrs esse cara é muito bom k'


def converte_abrevia(texto):
    novo_texto = []
    for w in texto.split():
        if w in abrevia:
            novo_texto.append(abrevia[w])
        else:
            novo_texto.append(w)

    return " ".join(novo_texto)

print(f'Original: {texto}')
print(f'Convertido: {converte_abrevia(texto)}')

Original: rsrs esse cara é muito bom k
Convertido: risos esse cara é muito bom kkk


In [94]:
df_text['texto_sem_abrevia'] = df_text['texto_sem_quebra'].apply(converte_abrevia)
df_text

,texto,texto_minusculo,texto_sem_arroba,texto_sem_quebra,texto_sem_abrevia
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ..."
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário eslu em caruaru,boticário eslu em caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkk
...,...,...,...,...,...
4820,accordes da oBoticário,accordes da oboticário,accordes da oboticário,accordes da oboticário,accordes da oboticário
4821,@LulaOficial Vc é um fracasso!,@lulaoficial vc é um fracasso!,vc é um fracasso!,vc é um fracasso!,você é um fracasso!
4822,"Desculpa, mas se você diz que é de Realengo e ...","desculpa, mas se você diz que é de realengo e ...","desculpa, mas se você diz que é de realengo e ...","desculpa, mas se você diz que é de realengo e ...","desculpa, mas se você diz que é de realengo e ..."
4823,o studiar e diventar uomini o volar a Empoli p...,o studiar e diventar uomini o volar a empoli p...,o studiar e diventar uomini o volar a empoli p...,o studiar e diventar uomini o volar a empoli p...,o studiar e diventar uomini o volar a empoli p...


## Removendo stopwords

Irei fazer a remoção das stop words. Para aqueles que não estão familiarizados com o conceito,nas palavras do www.clubedoportugues.com.br/stop-words/ "Stop word significa a palavra que pode ser totalmente suprimida, omitida ou ocultada na hora de fazer uma busca na internet sem que o sentido do que se quer encontrar seja perdido. Isso acontece, porque simplesmente as funções semânticas dessas palavras não se mostram tão determinantes para garantir resultados eficazes nos buscadores".

No conjunto de stopwords fornecidas pelo biblioteca nkl contém duas palavras de negação, irei retira-las pois essas palavras podem mudar o significado das frases.

In [95]:

def remove_stopwords(text, lingua='portuguese'):
    '''func para remover stopwords'''
    stop_words = set(stopwords.words(lingua))
    stop_words.remove('não')
    stop_words.remove('nem')
    words = text.split()
    words_sem_sw = [word for word in words if word not in stop_words]
    return ' '.join(words_sem_sw)

df_text['texto_sem_sw'] = df_text['texto_sem_abrevia'].apply(remove_stopwords)
df_text.head()


,texto,texto_minusculo,texto_sem_arroba,texto_sem_quebra,texto_sem_abrevia,texto_sem_sw
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente amo perfumes egeo boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 boticário amo ""dark mint"" put..."
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar boticário tia perigo sai parcelando pe...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário eslu em caruaru,boticário eslu em caruaru,boticário eslu caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkk,uso natura boticário feliz vida kkk


## Removendo URLS

In [96]:
#localziadno textos que tem urls
df_urls = df_text[df_text['texto_sem_sw'].str.contains('https', na = False)]
df_urls.head()

,texto,texto_minusculo,texto_sem_arroba,texto_sem_quebra,texto_sem_abrevia,texto_sem_sw
15,Pow depois de comprar uma bebida com gosto de ...,pow depois de comprar uma bebida com gosto de ...,pow depois de comprar uma bebida com gosto de ...,pow depois de comprar uma bebida com gosto de ...,pow depois de comprar uma bebida com gosto de ...,"pow comprar bebida gosto floratta boticário, p..."
23,Lembra? Boticário maquiou mulheres no dia do d...,lembra? boticário maquiou mulheres no dia do d...,lembra? boticário maquiou mulheres no dia do d...,lembra? boticário maquiou mulheres no dia do d...,lembra? boticário maquiou mulheres no dia do d...,lembra? boticário maquiou mulheres dia divórci...
34,😱😱😱😱😱😱\n\nColônia Quasar Next Star Wars The Ma...,😱😱😱😱😱😱\n\ncolônia quasar next star wars the ma...,😱😱😱😱😱😱\n\ncolônia quasar next star wars the ma...,😱😱😱😱😱😱 colônia quasar next star wars the mand...,😱😱😱😱😱😱 colônia quasar next star wars the manda...,😱😱😱😱😱😱 colônia quasar next star wars the manda...
35,😱😱😱😱😱😱\n\nDesodorante Colônia L'eau de Lily 75...,😱😱😱😱😱😱\n\ndesodorante colônia l'eau de lily 75...,😱😱😱😱😱😱\n\ndesodorante colônia l'eau de lily 75...,😱😱😱😱😱😱 desodorante colônia l'eau de lily 75ml...,😱😱😱😱😱😱 desodorante colônia l'eau de lily 75ml ...,😱😱😱😱😱😱 desodorante colônia l'eau lily 75ml - b...
41,Jackie Pinheiro: Rebecca se une ao Boticário p...,jackie pinheiro: rebecca se une ao boticário p...,jackie pinheiro: rebecca se une ao boticário p...,jackie pinheiro: rebecca se une ao boticário p...,jackie pinheiro: rebecca se une ao boticário p...,jackie pinheiro: rebecca une boticário lançar ...


In [97]:
df_urls.texto_sem_sw.loc[15]

'pow comprar bebida gosto floratta boticário, provei aqui, boa viu, podem ir, não gosto rexona v8 https://t.co/vvi0f7gh6e'

In [98]:
def remove_url(text):
    #Procura texto com o padrão especificado
    url_path = re.compile(r'https?\S+|www\S+')
    #substitui
    text_s_url = re.sub(url_path, '', text)
    return text_s_url

#testenado a funçao
text = df_text.texto_sem_sw.loc[15]
print(f'Texto original: {text}')
print(f'Texto sem url: {remove_url(text)}')

Texto original: pow comprar bebida gosto floratta boticário, provei aqui, boa viu, podem ir, não gosto rexona v8 https://t.co/vvi0f7gh6e
Texto sem url: pow comprar bebida gosto floratta boticário, provei aqui, boa viu, podem ir, não gosto rexona v8 


In [99]:
df_text['texto_sem_urls'] = df_text['texto_sem_sw'].apply(remove_url)
df_text.head()

,texto,texto_minusculo,texto_sem_arroba,texto_sem_quebra,texto_sem_abrevia,texto_sem_sw,texto_sem_urls
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 boticário amo ""dark mint"" put...","carai botica 214 boticário amo ""dark mint"" put..."
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário eslu em caruaru,boticário eslu em caruaru,boticário eslu caruaru,boticário eslu caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkk,uso natura boticário feliz vida kkk,uso natura boticário feliz vida kkk


## Removendo pontuação.

Irei remover as pontuações pois elas não agregam valor para as frases.

In [100]:
pont = string.punctuation

texto = 'Boa tarde, estão bem?!'
 
def remove_pont(texto):
    return texto.translate(str.maketrans('', '', pont))


remove_pont(texto)

'Boa tarde estão bem'

In [101]:

df_text['texto_sem_pont'] = df_text['texto_sem_urls'].apply(lambda x: remove_pont(x))
df_text.head()

,texto,texto_minusculo,texto_sem_arroba,texto_sem_quebra,texto_sem_abrevia,texto_sem_sw,texto_sem_urls,texto_sem_pont
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 boticário amo ""dark mint"" put...","carai botica 214 boticário amo ""dark mint"" put...",carai botica 214 boticário amo dark mint puta ...
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário eslu em caruaru,boticário eslu em caruaru,boticário eslu caruaru,boticário eslu caruaru,boticário eslu caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkk,uso natura boticário feliz vida kkk,uso natura boticário feliz vida kkk,uso natura boticário feliz vida kkk


## Convertendo emojis.


Por mais que os emojis sejam de fato texto, para o objetivo do projeto esses carinhas carregam muito contexo e carga sentimental e por isso optei em fazer a conversão dos emojis para palavras.

In [102]:
#identificando emojis
emoji_pattern = re.compile(r'[^\w\s,]')
df_urls = df_text[df_text['texto_sem_pont'].str.contains(emoji_pattern, na = False)]
df_urls.head()

,texto,texto_minusculo,texto_sem_arroba,texto_sem_quebra,texto_sem_abrevia,texto_sem_sw,texto_sem_urls,texto_sem_pont
5,"Ainda mais que quase ngm tem meu cheiro, impor...","ainda mais que quase ngm tem meu cheiro, impor...","ainda mais que quase ngm tem meu cheiro, impor...","ainda mais que quase ngm tem meu cheiro, impor...","ainda mais que quase ninguém tem meu cheiro, i...","ainda quase ninguém cheiro, importadissimo kkk...","ainda quase ninguém cheiro, importadissimo kkk...",ainda quase ninguém cheiro importadissimo kkk ...
8,O creme deleite da @boticario é o melhor ❤️,o creme deleite da @boticario é o melhor ❤️,o creme deleite da é o melhor ❤️,o creme deleite da é o melhor ❤️,o creme deleite da é o melhor ❤️,creme deleite melhor ❤️,creme deleite melhor ❤️,creme deleite melhor ❤️
10,Comprei 2 perfumes p mim na boticário. 😍,comprei 2 perfumes p mim na boticário. 😍,comprei 2 perfumes p mim na boticário. 😍,comprei 2 perfumes p mim na boticário. 😍,comprei 2 perfumes para mim na boticário. 😍,comprei 2 perfumes mim boticário. 😍,comprei 2 perfumes mim boticário. 😍,comprei 2 perfumes mim boticário 😍
12,Meus amigos minha irmã me deu um perfume da Ca...,meus amigos minha irmã me deu um perfume da ca...,meus amigos minha irmã me deu um perfume da ca...,meus amigos minha irmã me deu um perfume da ca...,meus amigos minha irmã me deu um perfume da ca...,"amigos irmã deu perfume carolina herrera, prim...","amigos irmã deu perfume carolina herrera, prim...",amigos irmã deu perfume carolina herrera prime...
13,"Com sono, com fome, mas muito cheirosa obg lin...","com sono, com fome, mas muito cheirosa obg lin...","com sono, com fome, mas muito cheirosa obg lin...","com sono, com fome, mas muito cheirosa obg lin...","com sono, com fome, mas muito cheirosa obrigad...","sono, fome, cheirosa obrigado linha bumbum bot...","sono, fome, cheirosa obrigado linha bumbum bot...",sono fome cheirosa obrigado linha bumbum botic...


In [103]:
#localizando
teste = df_text['texto_sem_pont'].loc[34]
teste

'😱😱😱😱😱😱 colônia quasar next star wars the mandalorian 50ml  boticário r 4890 preço promoção estoque oferta pode expirar rapidamente confira  '

In [104]:
#testando
def add_spaces_between_emojis(text):
    new_text = ''
    for c in text:
        if c in emoji.EMOJI_DATA:
            new_text += ' ' + c
        else:
            new_text += c
    return new_text.strip()


def convert_emoji(text): 
    emoji_name = emoji.demojize(text, language='pt').replace(",","").replace(":","")
    return emoji_name

convert_emoji(add_spaces_between_emojis(teste))

'rosto_gritando_de_medo rosto_gritando_de_medo rosto_gritando_de_medo rosto_gritando_de_medo rosto_gritando_de_medo rosto_gritando_de_medo colônia quasar next star wars the mandalorian 50ml  boticário r 4890 preço promoção estoque oferta pode expirar rapidamente confira'

In [105]:
#aplicando
df_text['texto_sem_emoji'] = df_text['texto_sem_pont'].apply(add_spaces_between_emojis)
df_text['texto_sem_emoji'] = df_text['texto_sem_emoji'].apply(convert_emoji)
df_text.head()

,texto,texto_minusculo,texto_sem_arroba,texto_sem_quebra,texto_sem_abrevia,texto_sem_sw,texto_sem_urls,texto_sem_pont,texto_sem_emoji
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 boticário amo ""dark mint"" put...","carai botica 214 boticário amo ""dark mint"" put...",carai botica 214 boticário amo dark mint puta ...,carai botica 214 boticário amo dark mint puta ...
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário eslu em caruaru,boticário eslu em caruaru,boticário eslu caruaru,boticário eslu caruaru,boticário eslu caruaru,boticário eslu caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkk,uso natura boticário feliz vida kkk,uso natura boticário feliz vida kkk,uso natura boticário feliz vida kkk,uso natura boticário feliz vida kkk


## Corrigindo a ortografia.

Utilizei a biblioteca spellcheker para fazer uma correção ortografica para que as palvras escritas de forma errada não sejam entendidas como palavras diferrentes.

In [106]:
spell = SpellChecker(language='pt')
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)
        
text = "vocse connsegue corigiir ezta frase?"
correct_spellings(text)

'voce consegue corrigir esta frase'

In [107]:
df_text['texto_final'] = df_text['texto_sem_emoji'].apply(convert_emoji)
df_text.head()

,texto,texto_minusculo,texto_sem_arroba,texto_sem_quebra,texto_sem_abrevia,texto_sem_sw,texto_sem_urls,texto_sem_pont,texto_sem_emoji,texto_final
0,Gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente eu amo os perfumes egeo da boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário,gente amo perfumes egeo boticário
1,"Carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 da boticário eu te amo ""dark ...","carai botica 214 boticário amo ""dark mint"" put...","carai botica 214 boticário amo ""dark mint"" put...",carai botica 214 boticário amo dark mint puta ...,carai botica 214 boticário amo dark mint puta ...,carai botica 214 boticário amo dark mint puta ...
2,Comprar o Boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar o boticário com a minha tia é um perig...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...,comprar boticário tia perigo sai parcelando pe...
3,@CentralEslo Boticário\nESLU EM CARUARU,@centraleslo boticário\neslu em caruaru,boticário\neslu em caruaru,boticário eslu em caruaru,boticário eslu em caruaru,boticário eslu caruaru,boticário eslu caruaru,boticário eslu caruaru,boticário eslu caruaru,boticário eslu caruaru
4,Uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkkkkkkkk,uso natura e boticário feliz da vida kkk,uso natura boticário feliz vida kkk,uso natura boticário feliz vida kkk,uso natura boticário feliz vida kkk,uso natura boticário feliz vida kkk,uso natura boticário feliz vida kkk


# Df final.

Depois de todo processamento do dataframe chegamos ao conjunto final.

In [108]:
df_texto = pd.read_csv(r'dados\tweets_classificados_raw.csv')
df_sentimento = pd.DataFrame(df_texto['sentimento'].copy())
df_sentimento

,sentimento
0,1
1,1
2,0
3,0
4,1
...,...
4820,0
4821,0
4822,0
4823,0


In [109]:
#concatenando
df_final = pd.concat([df_text['texto_final'], df_sentimento], axis=1)
df_final.head()

,texto_final,sentimento
0,gente amo perfumes egeo boticário,1
1,carai botica 214 boticário amo dark mint puta ...,1
2,comprar boticário tia perigo sai parcelando pe...,0
3,boticário eslu caruaru,0
4,uso natura boticário feliz vida kkk,1


In [110]:
#exemplo de tweet processado.

print(f'Original: {df_text.texto.loc[1877]}')
print(f'Tratados: {df_final.texto_final.loc[1877]}')

Original: boticário deveria me patrocinar, perdi a conta de quantos perfumes ja comprei kkkkkkk
Tratados: boticário deveria patrocinar perdi conta quantos perfumes ja comprei kkk


In [111]:
#exportando
df_final.to_csv(r'C:\Users\abraz\OneDrive\Área de Trabalho\projetos\analise_sentimento_tt\dados\tweets_tratados.csv', index=False)

In [112]:
df_final.loc[34]

texto_final    rosto_gritando_de_medo rosto_gritando_de_medo ...
sentimento                                                     0
Name: 34, dtype: object